## 1. Read the dataset

In [33]:
import pandas as pd

train_data_original = pd.read_csv("basket_data_by_date_train.csv")
test_data_original = pd.read_csv("basket_data_by_date_test.csv")

train_data_big = pd.read_csv("basket_data_by_date_train_big.csv")
test_data_big = pd.read_csv("basket_data_by_date_test_big.csv")

Check the structure and content of the dataset

In [34]:
print("Train Data Info:")
print(train_data_original.info())

print("\nTest Data Info:")
print(test_data_original.info())

Train Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   BillNo      40000 non-null  int64  
 1   Itemname    40000 non-null  object 
 2   Quantity    40000 non-null  int64  
 3   Date        40000 non-null  object 
 4   Price       40000 non-null  float64
 5   CustomerID  40000 non-null  int64  
 6   cost        40000 non-null  float64
dtypes: float64(2), int64(3), object(2)
memory usage: 2.1+ MB
None

Test Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   BillNo      10000 non-null  float64
 1   Itemname    10000 non-null  object 
 2   Quantity    10000 non-null  float64
 3   Date        10000 non-null  object 
 4   Price       10000 non-null  float64
 5   CustomerID  10000 non-

In [35]:
test_data_original.describe()

,BillNo,Quantity,Price,CustomerID,cost
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,545209.692000,3.672100,3.971454,15433.082900,11.840678
std,492.638789,2.639788,6.777596,1616.660727,13.086198
min,544398.000000,1.000000,0.160000,12352.000000,0.190000
25%,544795.000000,2.000000,1.650000,14227.000000,3.750000
50%,545181.000000,3.000000,2.950000,15358.000000,9.900000
75%,545664.000000,6.000000,4.950000,16931.000000,16.500000
max,546084.000000,10.000000,195.000000,18283.000000,290.000000


In [36]:
train_data_original.describe()

,BillNo,Quantity,Price,CustomerID,cost
count,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000
mean,540254.879225,3.487700,3.732165,15577.606525,11.097411
std,2380.444952,2.611766,5.711630,1730.347123,13.298301
min,536365.000000,1.000000,0.100000,12347.000000,0.140000
25%,538093.000000,1.000000,1.650000,14224.000000,3.300000
50%,540373.000000,2.000000,2.950000,15570.000000,7.950000
75%,542360.000000,6.000000,4.650000,17220.000000,15.800000
max,544398.000000,10.000000,295.000000,18283.000000,527.700000


In [37]:
negative_quantity_test = test_data_original[test_data_original['Quantity'] < 0]
print(negative_quantity_test)

Empty DataFrame
Columns: [BillNo, Itemname, Quantity, Date, Price, CustomerID, cost]
Index: []


In [38]:
negative_quantity_train = train_data_original[train_data_original['Quantity'] < 0]
print(negative_quantity_train)

Empty DataFrame
Columns: [BillNo, Itemname, Quantity, Date, Price, CustomerID, cost]
Index: []



Based on the information provided about the data, it can be seen that both the training and test sets contain seven columns, which are:

BillNo, Itemname, Quantity, Date, Price, CustomerID, cost

## 2. Pre-process the data

#### 2.1 Clean the missing value

In [39]:
# check if there are missing values in the train set
print("Train Data Missing Values:")
print(train_data_original.isnull().sum())

# check if there are missing values in the test set
print("\nTest Data Missing Values:")
print(test_data_original.isnull().sum())

Train Data Missing Values:
BillNo        0
Itemname      0
Quantity      0
Date          0
Price         0
CustomerID    0
cost          0
dtype: int64

Test Data Missing Values:
BillNo        30000
Itemname      30000
Quantity      30000
Date          30000
Price         30000
CustomerID    30000
cost          30000
dtype: int64


The data shows that the dataset of Test Data contains a large number of missing values, which is 30,000 in number. So I will deal with these missing values.

In [40]:
test_data_original.tail(20)

,BillNo,Itemname,Quantity,Date,Price,CustomerID,cost
39980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39981,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39982,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39983,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39984,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39985,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39986,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39987,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39988,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39989,NaN,NaN,NaN,NaN,NaN,NaN,NaN


From here we can see that the missing values are all at the tail of the dataset. So I'm going to delete all the data after 10000.

In [41]:
# 删除从第10001行开始的缺失值
test_data_clean = test_data_original.iloc[:10000]

# 检查删除缺失值后的数据集大小
print("The size after cleaned the missing values", test_data_clean.shape)

The size after cleaned the missing values (10000, 7)


See if there is any missing values at the end of the dataset

In [43]:
test_data_clean.tail(10)

,BillNo,Itemname,Quantity,Date,Price,CustomerID,cost
9990,546084.0,JUMBO STORAGE BAG SKULLS,10.0,9/03/2011 11:28,1.95,14112.0,19.50
9991,546084.0,SET 3 SONG BIRD PAPER EGGS ASSORTED,6.0,9/03/2011 11:28,2.95,14112.0,17.70
9992,546084.0,COFFEE MUG APPLES DESIGN,6.0,9/03/2011 11:28,2.55,14112.0,15.30
9993,546084.0,RED RETROSPOT SMALL MILK JUG,6.0,9/03/2011 11:28,2.55,14112.0,15.30
9994,546084.0,JUMBO BAG RED RETROSPOT,10.0,9/03/2011 11:28,1.95,14112.0,19.50
9995,546084.0,EASTER TIN CHICKS PINK DAISY,6.0,9/03/2011 11:28,1.65,14112.0,9.90
9996,546084.0,CREAM CUPID HEARTS COAT HANGER,4.0,9/03/2011 11:28,4.15,14112.0,16.60
9997,546084.0,RED RETROSPOT BOWL,8.0,9/03/2011 11:28,1.25,14112.0,10.00
9998,546084.0,RED RETROSPOT CAKE STAND,1.0,9/03/2011 11:28,10.95,14112.0,10.95
9999,546084.0,RED RETROSPOT CHARLOTTE BAG,10.0,9/03/2011 11:28,0.85,14112.0,8.50


In [44]:
test_data_clean.describe()

,BillNo,Quantity,Price,CustomerID,cost
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,545209.692000,3.672100,3.971454,15433.082900,11.840678
std,492.638789,2.639788,6.777596,1616.660727,13.086198
min,544398.000000,1.000000,0.160000,12352.000000,0.190000
25%,544795.000000,2.000000,1.650000,14227.000000,3.750000
50%,545181.000000,3.000000,2.950000,15358.000000,9.900000
75%,545664.000000,6.000000,4.950000,16931.000000,16.500000
max,546084.000000,10.000000,195.000000,18283.000000,290.000000


In [45]:
test_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   BillNo      10000 non-null  float64
 1   Itemname    10000 non-null  object 
 2   Quantity    10000 non-null  float64
 3   Date        10000 non-null  object 
 4   Price       10000 non-null  float64
 5   CustomerID  10000 non-null  float64
 6   cost        10000 non-null  float64
dtypes: float64(5), object(2)
memory usage: 547.0+ KB


Now we can see the missing values that are in the test_data_original dataset has successfully removed.

Because in the first part, I have checked that if the dataset contains any quantity of values < 0. The result shows that no value is below 0, which is reasonable, so we don't need to deal with it.

Since we are using collaborative filtering to generate system recommendations. It mainly rely on users' ratings or behaviors of items. 'BillNo', 'Date', 'Price' and 'Cost' are not very useful to us, because these features do not provide information related to user behavior. Thus, I will remove these columns.

#### 2.2 Remove not related columns

In [46]:
# 删除指定的列
train_data = train_data_original.drop(columns=['BillNo', 'Date', 'Price', 'cost'])
test_data = test_data_clean.drop(columns=['BillNo', 'Date', 'Price', 'cost'])

In [47]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Itemname    40000 non-null  object
 1   Quantity    40000 non-null  int64 
 2   CustomerID  40000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 937.6+ KB


In [48]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Itemname    10000 non-null  object 
 1   Quantity    10000 non-null  float64
 2   CustomerID  10000 non-null  float64
dtypes: float64(2), object(1)
memory usage: 234.5+ KB


#### 2.3 customer-item matrix

Now I need to convert the data into the form of customer-item matrix

In [49]:
len(train_data.CustomerID.unique()), len(train_data.Itemname.unique())

(1391, 2523)

In [50]:
# create a pivot dataframe with index as a userid and columns as a movieid

# 确定唯一的用户和项目标识符列表
unique_users = train_data['CustomerID'].unique()
unique_items = train_data['Itemname'].unique()

# 创建一个空的用户-项目矩阵
user_item_matrix = pd.DataFrame(index=unique_users, columns=unique_items)

# 遍历训练数据集，填充用户-项目矩阵
for index, row in train_data.iterrows():
    user_id = row['CustomerID']
    item_id = row['Itemname']
    rating = row['Quantity']
    user_item_matrix.loc[user_id, item_id] = rating

user_item_matrix.fillna(0, inplace=True)

# 仅打印前10行和前10列
print(user_item_matrix.iloc[:10, :10].to_string())

       KNITTED UNION FLAG HOT WATER BOTTLE  GLASS STAR FROSTED T-LIGHT HOLDER  WHITE HANGING HEART T-LIGHT HOLDER  RED WOOLLY HOTTIE WHITE HEART.  SET 7 BABUSHKA NESTING BOXES  CREAM CUPID HEARTS COAT HANGER  WHITE METAL LANTERN  HAND WARMER UNION JACK  HAND WARMER RED POLKA DOT  HOME BUILDING BLOCK WORD
17850                                    6                                  6                                   6                               6                             4                               8                    6                       6                          6                         0
13047                                    0                                  0                                   0                               0                             0                               0                    0                       0                          0                         3
14688                                    0                                  0     

/var/folders/n3/ddr1nygd4b11p0zv8kwtqx3w0000gn/T/ipykernel_16907/3377891989.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  user_item_matrix.fillna(0, inplace=True)


### 3. Calculate the similarity

In [ ]:
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation

# 计算用户之间的相似性
user_similarity = 1 - pairwise_distances(user_item_matrix.values, metric='cosine')
user_similarity_df = pd.DataFrame(user_similarity)

user_similarity_df.index = train_data.userId.unique()
user_similarity_df.columns = train_data.userId.unique()
user_similarity_df.head()